# `QuantumPioneer/databases` Demo
This notebook demonstrates how to interact with the `QuantumPioneer` databases.

See inline comments for more information.

In [1]:
import numpy as np

# both pandas and polars can read parquet files - pick whichever you prefer!
# there are advantages and disadvantages to both
import pandas as pd
import polars as pl

import pyarrow as pa
# this library interacts with the parquet format directly, and both pandas and polars can use it too
import pyarrow.parquet as pq

# schema = layout of the database (what are the datatypes, etc.)
# the schema for the quantumpioneer databases are stored in databases.schema and vary depending on the type of data
from databases.schema import DLPNO_SCHEMA, DFT_SCHEMA

## DLPNO

In [2]:
# set this to match wherever you have the database file located
DLPNO_DATABASE_FPATH = "QuantumPioneer_v1_DLPNO.parquet"

In [3]:
# just open and read the entire dataset (very practical with the DLPNO data, which is small), which will be slow with pandas
df = pd.read_parquet(
    DLPNO_DATABASE_FPATH,
    schema=DLPNO_SCHEMA,  # pandas will try and guess this on its own if you don't provide it - it gets it right, but is slower
)
df.head(4)

,source,route_section,charge,multiplicity,energy,run_time,input_coordinates,dipole_au
0,/data1/groups/co2_capture/reactant_product_cal...,uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J ...,0,2,-400.826710,995.0,"[[1.966178, 1.124096, -0.127534], [0.66917, 1....",1.19932
1,/data1/groups/co2_capture/reactant_product_cal...,uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J ...,0,2,-522.304109,3716.0,"[[3.793948, 0.51694, -0.021986], [2.578084, 0....",0.52795
2,/data1/groups/co2_capture/reactant_product_cal...,uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J ...,0,2,-422.263499,1073.0,"[[-1.503284, -1.357277, -0.49672], [-0.952133,...",0.30215
3,/data1/groups/co2_capture/reactant_product_cal...,uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J ...,0,2,-454.045509,990.0,"[[-1.905391, -1.469124, -0.652625], [-0.866371...",1.55913


In [4]:
# from here you can do all of your usual pandas manipulations
df.iloc[0][["route_section", "input_coordinates"]].to_list()

['uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J cc-pvqz/c cc-pvqz-f12-cabs RIJCOSX NormalSCF NormalPNO',
 array([array([ 1.966178,  1.124096, -0.127534]),
        array([0.66917 , 1.414874, 0.013796]),
        array([-0.144905,  0.22587 ,  0.479877]),
        array([-1.324741, -0.093627, -0.41136 ]),
        array([-2.652683,  0.470902, -0.158743]),
        array([-2.352086, -0.868398,  0.16531 ]),
        array([ 0.897242, -0.899951,  0.499408]),
        array([ 1.521856, -1.190401, -0.772729]),
        array([ 2.222622, -0.324108,  0.166081]),
        array([ 2.746526,  1.814837, -0.449243]),
        array([ 0.21828 ,  2.389503, -0.184036]),
        array([-0.532997,  0.393642,  1.499854]),
        array([-1.055263, -0.374274, -1.438704]),
        array([-3.350308,  0.617562, -0.992022]),
        array([-2.797654,  1.146153,  0.694767]),
        array([ 0.737028, -1.760108,  1.157542]),
        array([ 3.189127, -0.669057,  0.545677])], dtype=object)]

In [5]:
# you can reduce the amount of memory consumed by only loading the columns that you care about using columns=...
df = pd.read_parquet(DLPNO_DATABASE_FPATH, columns=["source", "energy"])
df.head(4)

,source,energy
0,/data1/groups/co2_capture/reactant_product_cal...,-400.826710
1,/data1/groups/co2_capture/reactant_product_cal...,-522.304109
2,/data1/groups/co2_capture/reactant_product_cal...,-422.263499
3,/data1/groups/co2_capture/reactant_product_cal...,-454.045509


In [6]:
# and you filter out specific rows _when reading_ the database to further reduce memory consumption (and speed things up)
# these statements can be complex, but the pandas docs explain it well:
# https://pandas.pydata.org/docs/reference/api/pandas.read_parquet.html
df = pd.read_parquet(
    DLPNO_DATABASE_FPATH,
    # SKIP rows where...
    filters=[
        [
            (  # multiplicity is equal to 1
                "multiplicity",
                "=",
                1,
            ),  # AND
            (  # energy is less than -500
                "energy",
                "<",
                -500,
            ),
        ],
        [  # OR
            (  # these two specific files
                "source",
                "not in",
                (
                    "/data1/groups/co2_capture/reactant_product_calculation/ts_nho_round1/output/DLPNO_sp_f12/outputs/outputs_146/146857.log",
                    "/data1/groups/co2_capture/reactant_product_calculation/ts_nho_round1/output/DLPNO_sp_f12/outputs/outputs_146/146989.log",
                ),
            ),
        ],
    ],
)
df.head(4)

,source,route_section,charge,multiplicity,energy,run_time,input_coordinates,dipole_au
0,/data1/groups/co2_capture/reactant_product_cal...,uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J ...,0,2,-400.826710,995.0,"[[1.966178, 1.124096, -0.127534], [0.66917, 1....",1.19932
1,/data1/groups/co2_capture/reactant_product_cal...,uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J ...,0,2,-522.304109,3716.0,"[[3.793948, 0.51694, -0.021986], [2.578084, 0....",0.52795
2,/data1/groups/co2_capture/reactant_product_cal...,uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J ...,0,2,-422.263499,1073.0,"[[-1.503284, -1.357277, -0.49672], [-0.952133,...",0.30215
3,/data1/groups/co2_capture/reactant_product_cal...,uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J ...,0,2,-454.045509,990.0,"[[-1.905391, -1.469124, -0.652625], [-0.866371...",1.55913


In [7]:
# you can also filter out based on one row without actually loading it
df = pd.read_parquet(DLPNO_DATABASE_FPATH, filters=[("run_time", "<", 100)], columns=["source", "charge"])
df.head(4)

,source,charge
0,/data1/groups/co2_capture/reactant_product_cal...,0
1,/data1/groups/co2_capture/reactant_product_cal...,0
2,/data1/groups/co2_capture/reactant_product_cal...,0
3,/data1/groups/co2_capture/reactant_product_cal...,0


In [8]:
# you can run everything from above using polars as well, and in my experience it uses less memory and is faster
df = pl.read_parquet(DLPNO_DATABASE_FPATH)
df.head(4)

source,route_section,charge,multiplicity,energy,run_time,input_coordinates,dipole_au
str,str,u8,u8,f64,u32,list[list[f64]],f32
"""/data1/groups/co2_capture/reac…","""uHF UNO DLPNO-CCSD(T)-F12D cc-…",0,2,-400.82671,995,"[[1.966178, 1.124096, -0.127534], [0.66917, 1.414874, 0.013796], … [3.189127, -0.669057, 0.545677]]",1.19932
"""/data1/groups/co2_capture/reac…","""uHF UNO DLPNO-CCSD(T)-F12D cc-…",0,2,-522.304109,3716,"[[3.793948, 0.51694, -0.021986], [2.578084, 0.203352, 0.838065], … [-1.554274, 2.973256, -0.233796]]",0.52795
"""/data1/groups/co2_capture/reac…","""uHF UNO DLPNO-CCSD(T)-F12D cc-…",0,2,-422.263499,1073,"[[-1.503284, -1.357277, -0.49672], [-0.952133, -0.060015, 0.07242], … [2.290489, 0.453102, 0.719722]]",0.30215
"""/data1/groups/co2_capture/reac…","""uHF UNO DLPNO-CCSD(T)-F12D cc-…",0,2,-454.045509,990,"[[-1.905391, -1.469124, -0.652625], [-0.866371, -0.836593, -0.159013], … [-0.069671, 2.402362, -0.805667]]",1.55913


In [9]:
# a notable difference is that polars sets `memory_map=True` by default (pandas supports it, but is False and accessible via kwarg only)

In [10]:
# to pass filters to polars, you have to use the `pyarrow_options` argument (polars only supports limiting the number of rows in
# in sequential order via `n_rows`)
df = pl.read_parquet(
    DLPNO_DATABASE_FPATH,
    columns=["source", "charge"],
    pyarrow_options=dict(
        filters=[("run_time", "<", 100)],
        schema=DLPNO_SCHEMA,
    ),
)
df.head(4)

source,charge
str,u8
"""/data1/groups/co2_capture/reac…",0
"""/data1/groups/co2_capture/reac…",0
"""/data1/groups/co2_capture/reac…",0
"""/data1/groups/co2_capture/reac…",0


In [11]:
# ...or just use polars other functions
df = (
    pl.scan_parquet(
        DLPNO_DATABASE_FPATH,
    )  # opens the file, but does not actually read it (LazyFrame)
    .filter(
        pl.col("run_time") < 100,
    )  # sets up our filters, but still does not run the query
    .select(pl.col("source"), pl.col("energy"))
    .collect()  # actually runs the query
)
df.head(4)

source,energy
str,f64
"""/data1/groups/co2_capture/reac…",-623.790852
"""/data1/groups/co2_capture/reac…",-170.225353
"""/data1/groups/co2_capture/reac…",-134.31814
"""/data1/groups/co2_capture/reac…",-515.612084


In [12]:
# the final option is to interact with the data via pyarrow directly, which takes all the same arguments as before but in a slightly
# different setup - this is the single fastest way to read the data
table = pq.ParquetDataset(DLPNO_DATABASE_FPATH, schema=DLPNO_SCHEMA, filters=[("run_time", "<", 100)]).read(columns=["source", "energy"])
df = table.to_pandas()
df.head(4)

,source,energy
0,/data1/groups/co2_capture/reactant_product_cal...,-623.790852
1,/data1/groups/co2_capture/reactant_product_cal...,-170.225353
2,/data1/groups/co2_capture/reactant_product_cal...,-134.318140
3,/data1/groups/co2_capture/reactant_product_cal...,-515.612084


## DFT
All of the above applies for the DFT data also - this will just demonstrate some of the fields that are specific to the DFT data, as well as limiting the number of rows (since this dataset is much bigger).

This section will just use `pyarrow.parquet` since this dataset is more complicated and this library can do a lot more "nuts and bolts" interactions with the data.
Everything shown could _probably_ be done with pandas or polars, but not as easily.

In [13]:
# set this to match wherever you have the database file located
DFT_DATABASE_FPATH = "QuantumPioneer_v1_DFT.parquet"

In [14]:
# look at the information about the database
f = pq.ParquetFile(DFT_DATABASE_FPATH)
f.metadata

  created_by: parquet-cpp-arrow version 16.1.0
  num_columns: 26
  num_rows: 453472
  num_row_groups: 886
  format_version: 2.6
  serialized_size: 3008687

In [15]:
# many rows in this data! just load some of them (see https://stackoverflow.com/a/69888274)
first_ten_rows = next(f.iter_batches(batch_size=64))
df = pa.Table.from_batches([first_ten_rows]).to_pandas()
df.head(4)

,source,route_section,charge,multiplicity,max_steps,normal_termination,cpu_time,wall_time,e0_h,hf,...,aniso_polarizability_au,iso_polarizability_au,scf,dipole_moment_debye,frequencies,mulliken_charges_summed,frequency_modes,xyz,std_xyz,std_forces
0,/data1/groups/RMG/Projects/Hao-Wei-Oscar-Yunsi...,"P opt=(calcfc,maxcycle=128,noeig,nomicro,carte...",0,2,100,True,2374,614,-382.633530,-382.760525,...,18.4085,62.8190,"[382.758210298, 382.759560141, 382.760381356, ...","[1.8824, 1.7059, 0.6066]","[131.7364, 188.0713, 194.6236, 263.776, 273.62...","[[1.0, 0.234094], [2.0, -0.375229], [3.0, 0.37...","[[[1.0, 6.0, -0.07, -0.1, -0.16], [2.0, 6.0, -...","[[1.0, 6.0, 0.0, 0.178454, 1.90411, 0.117548],...","[[[1.0, 6.0, 0.0, -0.175993, 1.912209, -0.1176...","[[[1.0, 6.0, -2.303e-05, 0.000148725, 7.95e-06..."
1,/data1/groups/RMG/Projects/Hao-Wei-Oscar-Yunsi...,"P opt=(calcall,maxcycle=64,noeig,nomicro,carte...",0,2,64,True,5603,1415,-319.766421,-319.876823,...,37.2826,58.9151,"[319.870024548, 319.873712193, 319.875876296, ...","[-2.9536, 6.6691, 0.0]","[125.7135, 225.1377, 243.6628, 275.6355, 307.2...","[[1.0, 0.177478], [2.0, -0.11589], [3.0, -0.08...","[[[1.0, 6.0, -0.0, -0.0, -0.02], [2.0, 6.0, -0...","[[1.0, 6.0, 0.0, -2.04409, -0.600169, 0.01305]...","[[[1.0, 6.0, 0.0, -2.050841, -0.610164, -0.000...","[[[1.0, 6.0, 0.001360017, 0.000413491, -5.4464..."
2,/data1/groups/RMG/Projects/Hao-Wei-Oscar-Yunsi...,"P opt=(calcfc,maxcycle=128,noeig,nomicro,carte...",0,2,120,True,9251,2448,-423.099887,-423.281993,...,14.4899,74.3067,"[423.274519549, 423.277512792, 423.280067289, ...","[0.6979, 0.8904, 0.3061]","[61.6212, 178.6859, 229.9573, 253.2535, 273.82...","[[1.0, 0.178076], [2.0, -0.425597], [3.0, 0.11...","[[[1.0, 6.0, -0.08, 0.02, -0.05], [2.0, 6.0, 0...","[[1.0, 6.0, 0.0, 0.243366, 2.083137, 0.689603]...","[[[1.0, 6.0, 0.0, -0.459041, 2.197114, -0.2423...","[[[1.0, 6.0, -4.106e-06, 1.1987e-05, -2.732e-0..."
3,/data1/groups/RMG/Projects/Hao-Wei-Oscar-Yunsi...,"P opt=(calcall,maxcycle=64,noeig,nomicro,carte...",0,2,64,True,8237,537,-303.770502,-303.893091,...,46.1818,63.9089,"[303.886122623, 303.88988402, 303.892570737, 3...","[3.0462, 0.8742, 0.0015]","[128.1369, 165.4973, 216.2848, 249.2191, 292.2...","[[1.0, 0.154953], [2.0, -0.118076], [3.0, -0.0...","[[[1.0, 6.0, -0.0, 0.0, -0.03], [2.0, 6.0, -0....","[[1.0, 6.0, 0.0, -1.37189, -1.175717, -0.07927...","[[[1.0, 6.0, 0.0, 1.994908, -0.843335, 0.00100...","[[[1.0, 6.0, -0.000275241, 6.5551e-05, 1.2646e..."


In [16]:
# we can also apply the many filters from above on this data
table = pq.ParquetDataset(
    DFT_DATABASE_FPATH,
    schema=DFT_SCHEMA,
    filters=[("hf", "!=", np.nan)],  # skip rows where hf is missing
    memory_map=True,  # reduce memory usage by delaying reads into memory
).read(columns=["multiplicity", "hf"])

for batch in table.to_batches(512):
    print(batch.to_pandas().head(4))
    break

   multiplicity          hf
0             2 -382.760525
1             2 -319.876823
2             2 -423.281993
3             2 -303.893091


In [17]:
# the last thing I will point to is the pyarrow scanner, which makes a lot of these operations easy too
# https://arrow.apache.org/docs/python/generated/pyarrow.dataset.Scanner.html
# This part of the pyarrow API is currently experimental, so it might change, but it is very useful

ds = pa.dataset.dataset(DFT_DATABASE_FPATH, schema=DFT_SCHEMA)
s = pa.dataset.Scanner.from_dataset(
    ds,
    columns=["charge", "multiplicity", "xyz"],
    filter=(pa.compute.field("hf") != pa.compute.scalar(np.nan)),  # must use pyarrow.compute syntax instead of plain strings...
    batch_size=5_096,  # reduce this to fit in your memory limitations
)
for batch in s.to_batches():
    print(batch.to_pandas()['xyz'].head(4))
    break

0    [[1.0, 6.0, 0.0, 0.178454, 1.90411, 0.117548],...
1    [[1.0, 6.0, 0.0, -2.04409, -0.600169, 0.01305]...
2    [[1.0, 6.0, 0.0, 0.243366, 2.083137, 0.689603]...
3    [[1.0, 6.0, 0.0, -1.37189, -1.175717, -0.07927...
Name: xyz, dtype: object
